# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft as sy

In [6]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [31]:
bob = sy.VirtualWorker(hook, id="bob")

In [32]:
bob._objects

{75141757971: tensor([2, 3, 1, 8])}

In [33]:
x = th.tensor([1,2,3,4,5])

In [34]:
x = x.send(bob)

In [35]:
bob._objects

{75141757971: tensor([2, 3, 1, 8]), 87353135869: tensor([1, 2, 3, 4, 5])}

In [36]:
x.location

<VirtualWorker id:bob #objects:2>

In [37]:
x.id_at_location

87353135869

In [38]:
x.id

40677404791

In [39]:
x.owner

<VirtualWorker id:me #objects:0>

In [40]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [41]:
x

(Wrapper)>[PointerTensor | me:40677404791 -> bob:87353135869]

In [42]:
x = x.get()   # Lo quita de bob
x

tensor([1, 2, 3, 4, 5])

In [43]:
bob._objects


{75141757971: tensor([2, 3, 1, 8])}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [21]:
# try this project here!

In [44]:
alice = sy.VirtualWorker(hook, id="alice")

In [45]:
mytensor = th.tensor([2, 3, 1 ,8])

In [46]:
sent_tensor = mytensor.send(bob, alice)

In [47]:
sent_tensor.float().mean()


(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:753020122 -> bob:30744358020]
	-> (Wrapper)>[PointerTensor | me:15161226237 -> alice:72823904668]

In [48]:
sent_tensor.float().mean().get()

[tensor(3.5000), tensor(3.5000)]

In [49]:
sent_tensor.id_at_location

In [50]:
sent_tensor.owner


<VirtualWorker id:me #objects:0>

In [51]:
sent_tensor.get()

[tensor([2, 3, 1, 8]), tensor([2, 3, 1, 8])]

# Lesson: Introducing Remote Arithmetic

In [52]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [53]:
x

(Wrapper)>[PointerTensor | me:21246835811 -> bob:99295648149]

In [54]:
y

(Wrapper)>[PointerTensor | me:1162244024 -> bob:58299156468]

In [55]:
z = x + y

In [56]:
z

(Wrapper)>[PointerTensor | me:88756051171 -> bob:31617812722]

In [57]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [58]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:77466437760 -> bob:47961980326]

In [59]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [60]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [61]:
z = (x + y).sum()

In [62]:
z.backward()

(Wrapper)>[PointerTensor | me:77777700733 -> bob:27889547454]

In [63]:
x = x.get()

In [64]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [65]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [ ]:
# try this project here!

In [ ]:
'''
const config = {
    binaryThresh: 0.5,
    hiddenLayers: [3],     // array of ints for the sizes of the hidden layers in the network
    activation: 'sigmoid',  // supported activation types: ['sigmoid', 'relu', 'leaky-relu', 'tanh'],
    leakyReluAlpha: 0.01   // supported for activation type 'leaky-relu'
};

// create a simple feed forward neural network with backpropagation
const net = new brain.NeuralNetwork(config);

net.train([{input: [0, 0], output: [0]},
           {input: [0, 1], output: [1]},
           {input: [1, 0], output: [1]},
           {input: [1, 1], output: [0]}]);

const output = net.run([1, 0]);  // [0.987]
'''


In [66]:
input = th.tensor([ [0., 0.], [0., 1.], [1., 0.], [1., 1.] ], requires_grad=True)
target = th.tensor([ [0.], [1.], [1.], [1.] ], requires_grad=True)
r_input = input.send(bob)
r_target = target.send(bob)

weights = th.tensor([[0.], [0.]], requires_grad=True)
r_weights = weights.send(bob)

for i in range(10):
    pred = r_input.mm(r_weights)    
    loss = ((pred - r_target)**2).sum()
    loss.backward()
    # print('pred: ', pred.get())
    r_weights.data.sub_(r_weights.grad * 0.1)
    r_weights.grad *= 0
    print(loss.get().data)

#test_pred    
pred = th.tensor([ [0., 0.], [0., 1.], [1., 0.], [1., 1.] ]).mm(r_weights.get())
pred

tensor(3.)
tensor(0.7600)
tensor(0.4016)
tensor(0.3443)
tensor(0.3351)
tensor(0.3336)
tensor(0.3334)
tensor(0.3333)
tensor(0.3333)
tensor(0.3333)


tensor([[0.0000],
        [0.6666],
        [0.6666],
        [1.3332]], grad_fn=<MmBackward>)

# Lesson: Garbage Collection and Common Errors


In [67]:
bob = bob.clear_objects()

In [68]:
bob._objects

{}

In [69]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [70]:
bob._objects

{}

In [71]:
del x

In [72]:
bob._objects

{}

In [73]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [74]:
bob._objects

{}

In [75]:
x = "asdf"

In [76]:
bob._objects

{}

In [77]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [78]:
x

(Wrapper)>[PointerTensor | me:15358425786 -> bob:10835670950]

In [79]:
bob._objects

{}

In [80]:
x = "asdf"

In [81]:
bob._objects

{}

In [82]:
del x

In [83]:
bob._objects

{}

In [84]:
bob = bob.clear_objects()
bob._objects

{}

In [85]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [86]:
bob._objects

{}

In [87]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [88]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:10600252949 -> bob:29946495886]
Tensor B: tensor([1, 1, 1, 1, 1])

In [89]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [90]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:0> while the other is on <VirtualWorker id:alice #objects:2>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [111]:
from torch import nn, optim

In [112]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [113]:
# A Toy Model
model = nn.Linear(2,1)

In [114]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [115]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(5.8429)
tensor(0.9575)
tensor(0.4101)
tensor(0.2525)
tensor(0.1649)
tensor(0.1089)
tensor(0.0723)
tensor(0.0483)
tensor(0.0324)
tensor(0.0219)
tensor(0.0149)
tensor(0.0102)
tensor(0.0071)
tensor(0.0049)
tensor(0.0035)
tensor(0.0024)
tensor(0.0017)
tensor(0.0013)
tensor(0.0009)
tensor(0.0007)


In [116]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [117]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [118]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [119]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [120]:
train()

KeyError: 'Object "13822423218" not found on worker!!!You just tried to interact with an object ID:13822423218 on <VirtualWorker id:bob #objects:0> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [32]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [101]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [102]:
x = x.send(alice)

In [103]:
bob._objects

{4843851482: tensor([1, 2, 3, 4, 5])}

In [104]:
alice._objects

{52538453696: (Wrapper)>[PointerTensor | alice:52538453696 -> bob:4843851482]}

In [105]:
y = x + x

In [107]:
y

(Wrapper)>[PointerTensor | me:98779283156 -> alice:72908441332]

In [83]:
bob._objects

{17166087971: tensor([1, 2, 3, 4, 5]),
 46165128435: tensor([ 2,  4,  6,  8, 10])}

In [84]:
alice._objects

{83495020895: (Wrapper)>[PointerTensor | alice:83495020895 -> bob:17166087971],
 72065210406: (Wrapper)>[PointerTensor | alice:72065210406 -> bob:46165128435]}

In [85]:
jon = sy.VirtualWorker(hook, id="jon")

In [86]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [87]:
bob._objects

{96657720227: tensor([1, 2, 3, 4, 5])}

In [88]:
alice._objects

{76523198928: (Wrapper)>[PointerTensor | alice:76523198928 -> bob:96657720227]}

In [89]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:76523198928 -> bob:96657720227]

In [90]:
bob._objects

{96657720227: tensor([1, 2, 3, 4, 5])}

In [91]:
alice._objects

{}

In [92]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [93]:
bob._objects

{}

In [94]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [95]:
bob._objects

{42037373897: tensor([1, 2, 3, 4, 5])}

In [96]:
alice._objects

{49357171101: (Wrapper)>[PointerTensor | alice:49357171101 -> bob:42037373897]}

In [97]:
del x

In [98]:
bob._objects

{}

In [99]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [108]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [109]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [112]:
x

(Wrapper)>[PointerTensor | me:36527443992 -> bob:50019340175]

In [110]:
bob._objects

{50019340175: tensor([1, 2, 3, 4, 5])}

In [111]:
alice._objects

{}

In [113]:
x.move(alice)

(Wrapper)>[PointerTensor | me:36527443992 -> alice:36527443992]

In [114]:
x

(Wrapper)>[PointerTensor | me:36527443992 -> alice:36527443992]

In [217]:
bob._objects

{}

In [218]:
alice._objects

{547575813: tensor([1, 2, 3, 4, 5])}

<VirtualWorker id:alice #objects:0>

In [121]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [122]:
x

(Wrapper)>[PointerTensor | me:97156655300 -> alice:25280123839]

In [123]:
bob._objects

{14376356676: tensor([1, 2, 3, 4, 5])}

In [124]:
alice._objects

{25280123839: (Wrapper)>[PointerTensor | alice:25280123839 -> bob:14376356676]}

In [125]:
x.remote_get()

(Wrapper)>[PointerTensor | me:97156655300 -> alice:25280123839]

In [126]:
bob._objects

{}

In [127]:
alice._objects

{25280123839: tensor([1, 2, 3, 4, 5])}

In [128]:
x.move(bob)

(Wrapper)>[PointerTensor | me:97156655300 -> bob:97156655300]

In [129]:
x

(Wrapper)>[PointerTensor | me:97156655300 -> bob:97156655300]

In [130]:
bob._objects

{97156655300: tensor([1, 2, 3, 4, 5])}

In [131]:
alice._objects

{}